## 准备数据

In [29]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [4]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [30]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        initializer = tf.keras.initializers.GlorotUniform()
        self.W1 = tf.Variable(initializer(shape=[28 * 28, 100]), trainable=True)
        self.b1 = tf.Variable(tf.zeros(100), trainable=True)
        self.W2 = tf.Variable(initializer(shape=[100, 10]), trainable=True)
        self.b2 = tf.Variable(tf.zeros(10), trainable=True)
        self.trainable_variables = [self.W1, self.W2, self.b1, self.b2]
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 28 * 28])
        x = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)  # 改用ReLU
        logits = tf.matmul(x, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [31]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [32]:
train_data, test_data = mnist_dataset()
for epoch in range(500):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3443182 ; accuracy 0.11721667
epoch 1 : loss 2.335508 ; accuracy 0.122033335
epoch 2 : loss 2.3268347 ; accuracy 0.12681666
epoch 3 : loss 2.3182936 ; accuracy 0.13223334
epoch 4 : loss 2.3098757 ; accuracy 0.13816667
epoch 5 : loss 2.3015761 ; accuracy 0.14455
epoch 6 : loss 2.2933855 ; accuracy 0.15088333
epoch 7 : loss 2.2853005 ; accuracy 0.15615
epoch 8 : loss 2.2773187 ; accuracy 0.16226667
epoch 9 : loss 2.2694316 ; accuracy 0.16808334
epoch 10 : loss 2.2616308 ; accuracy 0.17485
epoch 11 : loss 2.2539163 ; accuracy 0.18135
epoch 12 : loss 2.246285 ; accuracy 0.18745
epoch 13 : loss 2.2387328 ; accuracy 0.19383334
epoch 14 : loss 2.2312562 ; accuracy 0.19956666
epoch 15 : loss 2.223851 ; accuracy 0.20613334
epoch 16 : loss 2.2165124 ; accuracy 0.21271667
epoch 17 : loss 2.2092352 ; accuracy 0.22011666
epoch 18 : loss 2.2020178 ; accuracy 0.22725
epoch 19 : loss 2.1948593 ; accuracy 0.23411667
epoch 20 : loss 2.1877563 ; accuracy 0.24096666
epoch 21 : loss 2.1807

epoch 176 : loss 1.3224998 ; accuracy 0.72715
epoch 177 : loss 1.3184326 ; accuracy 0.72775
epoch 178 : loss 1.3143867 ; accuracy 0.72828335
epoch 179 : loss 1.3103628 ; accuracy 0.7287667
epoch 180 : loss 1.3063601 ; accuracy 0.7294
epoch 181 : loss 1.3023787 ; accuracy 0.7302
epoch 182 : loss 1.2984184 ; accuracy 0.7309833
epoch 183 : loss 1.2944791 ; accuracy 0.73178333
epoch 184 : loss 1.2905611 ; accuracy 0.7325
epoch 185 : loss 1.2866644 ; accuracy 0.73328334
epoch 186 : loss 1.2827888 ; accuracy 0.7341833
epoch 187 : loss 1.2789338 ; accuracy 0.7349833
epoch 188 : loss 1.2750987 ; accuracy 0.7356333
epoch 189 : loss 1.2712841 ; accuracy 0.7363833
epoch 190 : loss 1.26749 ; accuracy 0.73698336
epoch 191 : loss 1.2637165 ; accuracy 0.73793334
epoch 192 : loss 1.2599635 ; accuracy 0.73866665
epoch 193 : loss 1.2562307 ; accuracy 0.7391833
epoch 194 : loss 1.2525175 ; accuracy 0.73983335
epoch 195 : loss 1.2488244 ; accuracy 0.74058336
epoch 196 : loss 1.2451515 ; accuracy 0.7414833

epoch 350 : loss 0.8654184 ; accuracy 0.8089
epoch 351 : loss 0.8638357 ; accuracy 0.8092167
epoch 352 : loss 0.86226106 ; accuracy 0.80946666
epoch 353 : loss 0.86069465 ; accuracy 0.80978334
epoch 354 : loss 0.85913587 ; accuracy 0.80986667
epoch 355 : loss 0.857585 ; accuracy 0.81005
epoch 356 : loss 0.8560418 ; accuracy 0.81035
epoch 357 : loss 0.8545065 ; accuracy 0.81065
epoch 358 : loss 0.85297877 ; accuracy 0.81075
epoch 359 : loss 0.8514587 ; accuracy 0.81091666
epoch 360 : loss 0.8499462 ; accuracy 0.8111333
epoch 361 : loss 0.848441 ; accuracy 0.8114167
epoch 362 : loss 0.8469431 ; accuracy 0.81168336
epoch 363 : loss 0.84545237 ; accuracy 0.8118
epoch 364 : loss 0.84396875 ; accuracy 0.81205
epoch 365 : loss 0.8424926 ; accuracy 0.81233335
epoch 366 : loss 0.84102345 ; accuracy 0.8125833
epoch 367 : loss 0.8395616 ; accuracy 0.8127667
epoch 368 : loss 0.83810675 ; accuracy 0.8131
epoch 369 : loss 0.83665913 ; accuracy 0.81335
epoch 370 : loss 0.83521837 ; accuracy 0.8136
ep